In [1]:
! pip install transformers datasets evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.1 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=a3818134de5288b07e06e233f474d66bf79670d9060f47410fcd17906606e07a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dep

In [59]:
from datasets import load_dataset

dataset = load_dataset("sujayC66/text_summarization_512_length_1_2000",split="train[:1000]")

README.md:   0%|          | 0.00/416 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.48M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/320k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1683 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/297 [00:00<?, ? examples/s]

In [60]:
dataset

Dataset({
    features: ['content', 'summary'],
    num_rows: 1000
})

In [61]:
dataset = dataset.train_test_split(test_size=0.2)

In [62]:
dataset

DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 800
    })
    test: Dataset({
        features: ['content', 'summary'],
        num_rows: 200
    })
})

In [63]:
dataset["train"][0]

{'content': "Bengaluru, Jan 13 (IANS) A high-level delegation led by Heavy and Medium Industries Minister M.B. Patil will leave on January 14 to attend the World Economic Forum-2024 annual meet which will be held in Davos, Switzerland from January 15 to 19.Patil said that during the meeting, the delegation will draw the attention of businessmen over to the Karnataka government's business-friendly policies, evolved ecosystem, potential areas for investment, availability of human resources, priority to imbibe skills among students, and efficient single window system.He said that the delegation will hold all the meetings in Davos under the ‘Innovation Will Impact’ theme.“Our government aims to develop Karnataka as the leading manufacturing hub of Asia. We are also focussing on areas such as semiconductor, electric automotive, space and defense, clean energy, research and development and artificial intelligence,” the minister said.He said that the delegation’s aim is to attract more invest

##Model 1

In [64]:
from transformers import AutoTokenizer

checkpoint = "t5-small" #https://huggingface.co/t5-small
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [68]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["content"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [69]:
tokenized_text = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [70]:
tokenized_text

DatasetDict({
    train: Dataset({
        features: ['content', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 800
    })
    test: Dataset({
        features: ['content', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
})

In [71]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [72]:
!pip install evaluate

In [73]:
import evaluate

rouge = evaluate.load("rouge")

In [74]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [75]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [76]:
# prompt: huggignface login

!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineG

In [77]:
training_args = Seq2SeqTrainingArguments(
    output_dir="summarize_model",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.05,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_text["train"],
    eval_dataset=tokenized_text["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-77-88ade6c57073>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [78]:
#start training
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,1.323217,0.266600,0.148700,0.237700,0.236200,18.970000
2,No log,1.248057,0.275900,0.154800,0.245400,0.245000,18.970000
3,No log,1.219131,0.282100,0.161700,0.250300,0.250300,18.970000
4,No log,1.210833,0.282900,0.162600,0.251300,0.251200,18.970000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=200, training_loss=1.4317457580566406, metrics={'train_runtime': 148.6713, 'train_samples_per_second': 21.524, 'train_steps_per_second': 1.345, 'total_flos': 543058978406400.0, 'train_loss': 1.4317457580566406, 'epoch': 4.0})

In [86]:
content = "PALO ALTO - In a significant move to bolster its cloud services and artificial intelligence capabilities, Hewlett Packard Enterprise (NYSE:HPE) has entered into a definitive agreement to acquire Juniper Networks (NYSE:JNPR) at a purchase price of $40 per share, which equates to an equity value of approximately $14 billion. This strategic acquisition is set to enhance HPE's portfolio by integrating Juniper’s advanced Mist AI technology with the assets of HPE Aruba Networking.The merger, which is anticipated to be finalized between late 2024 and early 2025, is currently subject to customary closing conditions and regulatory approvals. Once completed, this deal is poised to position HPE as a player in the realm of secure unified cloud services and AI-native networking solutions. The move is seen as a response to the growing IT trends that emphasize artificial intelligence and hybrid cloud environments, with the goal of delivering comprehensive technology solutions that span from edge devices to cloud infrastructure.Rami Rahim, the current CEO of Juniper Networks, is set to lead the expanded networking sector within HPE after the transaction is completed. The financing strategy for this ambitious acquisition includes securing $14 billion in term loans, alongside plans for future refinancing to manage the investment effectively.This article was generated with the support of AI and reviewed by an editor. For more information see our T&C."

In [88]:
inputs = tokenizer(content, return_tensors="pt").input_ids

In [90]:
outputs = model.generate(inputs.cuda(), max_new_tokens=100, do_sample=False)

In [91]:
outputs

tensor([[    0, 32099,    12,  7464, 12170,   883,  3426,     7,    41, 23397,
            10,   683,   567,  5554,    61,    44,     3,     9,  1242,   594,
            13, 23853,   399,   698,     6,    84,     3, 25875,     7,    12,
            46,  8408,   701,    13,  3241, 26845,  2108,     5,    37,  6566,
            19,  1644,    12,  8726,  5481,   427,    31,     7,  3126,   364,
            11,  7353,  6123,  5644,    57,     3, 20030, 12170,   883,    22,
             7,  8306,    17,  7833,   748,    28,  5481,   427,  1533, 17309,
          3426,    53,     5,     1]], device='cuda:0')

In [92]:
#Decode the generated token ids back into text:
tokenizer.decode(outputs[0], skip_special_tokens=True)

"to acquire Juniper Networks (NYSE:JNPR) at a purchase price of $40 per share, which equates to an equity value of approximately $14 billion. The acquisition is expected to strengthen HPE's cloud services and artificial intelligence capabilities by integrating Juniper’s Mist AI technology with HPE Aruba Networking."